In [1]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.6MB/s 


In [0]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from google.colab import files

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

In [9]:
id = "1ocueYuEHk0KhxWSjq3w7du1se3yrnT7N"

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile("fd_stock_data.pkl")  

data_fd = pd.read_pickle("fd_stock_data.pkl")
data_fd.tail()

Attributes           Adj Close       Close        High         Low  \
Date       Symbols                                                   
2019-04-04 XYL       80.000000   80.000000   80.180000   79.480003   
           YUM      100.449997  100.449997  100.769997   99.940002   
           ZBH      126.559998  126.559998  127.279999  125.870003   
           ZION      47.680000   47.680000   47.779999   46.250000   
           ZTS      101.811356  101.980003  102.400002  100.639999   

Attributes                Open     Volume  Return10  F_return10  Adj Close_fd  
Date       Symbols                                                             
2019-04-04 XYL       79.660004   637700.0  0.009464    0.036625      5.593879  
           YUM      100.580002   860700.0  0.012703    0.028571     25.633876  
           ZBH      126.570000   817000.0 -0.021304   -0.051833     28.908942  
           ZION      46.250000  2259600.0  0.035396   -0.007970     10.836417  
           ZTS      102.190002  2475500.0  0.023485   -0.027319      7.783250

In [11]:
fd_columns = [x for x in data_fd.columns if x.find("_fd")!=-1]
fd_columns

['Adj Close_fd']

In [12]:
# Delete dates with less than 80% of stocks populated
data_fd.reset_index(inplace=True)
data_fd.set_index(['Date'], inplace=True)

data_fd_ = data_fd.loc[data_fd[fd_columns].groupby(level=0).count().min(axis=1) >= 
    0.8 * data_fd.groupby(level=0).agg({'Close': 'count'}).iloc[:,0], :]

data_fd_.reset_index(inplace=True)
data_fd_.set_index(['Date', 'Symbols'], inplace=True)

#Populate the rest of missing values with mean
data_fd_.loc[:,fd_columns] = data_fd_[fd_columns].groupby(level=1).transform(lambda x: x.fillna(x.mean()))

#Normalise
data_norm = data_fd_
data_norm.loc[:,fd_columns] = data_norm[fd_columns].groupby(level=1).transform(lambda x: (x - x.mean()) / x.std())
data_norm.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/loca

Attributes           Adj Close       Close        High         Low  \
Date       Symbols                                                   
2014-11-20 A         39.477184   41.259998   41.430000   40.560001   
           AAL       42.402725   44.160000   44.720001   43.200001   
           AAP      144.090805  145.190002  146.250000  144.080002   
           AAPL     108.010109  116.309998  116.860001  114.849998   
           ABBV      55.014709   65.190002   65.680000   64.930000   

Attributes                Open      Volume  Return10  F_return10  Adj Close_fd  
Date       Symbols                                                              
2014-11-20 A         40.560001   2303800.0 -0.002659    0.025206      0.201717  
           AAL       43.650002   9431400.0  0.023644    0.155118      0.053757  
           AAP      144.210007    485000.0 -0.002542    0.061919      0.839647  
           AAPL     114.910004  43395500.0  0.070010   -0.011263      0.166153  
           ABBV      65.040001   4975500.0  0.039547    0.069335      0.155458

In [14]:
data_norm.loc[pd.IndexSlice[:, 'AWK'], ['Adj Close', 'Adj Close_fd', 'Return10', 'F_return10']]

,Attributes,Adj Close,Adj Close_fd,Return10,F_return10
Date,Symbols,,,,
2014-11-20,AWK,47.380032,-1.087746,0.003270,-0.000383
2014-11-21,AWK,47.489037,-1.054464,-0.001528,0.007842
2014-11-24,AWK,47.443611,-1.078199,-0.012105,0.019721
2014-11-25,AWK,47.289192,-1.128054,-0.012519,0.008260
2014-11-26,AWK,47.634369,-1.026536,0.001719,0.003051
2014-11-28,AWK,48.188473,-0.890241,0.022749,-0.012818
2014-12-01,AWK,47.489037,-1.135078,0.007904,-0.011859
2014-12-02,AWK,47.779713,-1.033808,0.012122,-0.022434
2014-12-03,AWK,47.852379,-1.029639,0.006496,-0.017464


In [38]:
#Other transformations of source data
trans_columns_ = fd_columns

#Dictionary mapping suffix to transformation
trans = {
    #1. EWMA
    "_ewm60" : lambda x: x[trans_columns_].ewm(span=60).mean(),
    "_ewm10" : lambda x: x[trans_columns_].ewm(span=10).mean(),
    "_ewm26" : lambda x: x[trans_columns_].ewm(span=26).mean(),
    "_ewm12" : lambda x: x[trans_columns_].ewm(span=12).mean(),
    #2. SMA
    "_sma60" : lambda x: x[trans_columns_].rolling(window=60, min_periods=1).mean(),
    "_sma10" : lambda x: x[trans_columns_].rolling(window=10, min_periods=1).mean(),
    #3. Rolling SD
    "_std60" : lambda x: x[trans_columns_].rolling(window=60, min_periods=1).std(),
    "_std10" : lambda x: x[trans_columns_].rolling(window=10, min_periods=1).std(),
    #4. MACD
    "_macd": lambda x: x[[s + '_ewm12' for s in trans_columns_]].sub(x[[s + '_ewm26' for s in trans_columns_]].values)
} 

for suff, func in trans.items():
  data_norm[[c + suff for c in trans_columns_]] = data_norm.groupby(level=1).apply(func)
  
data_norm.tail()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


Attributes           Adj Close       Close        High         Low  \
Date       Symbols                                                   
2019-04-04 XYL       80.000000   80.000000   80.180000   79.480003   
           YUM      100.449997  100.449997  100.769997   99.940002   
           ZBH      126.559998  126.559998  127.279999  125.870003   
           ZION      47.680000   47.680000   47.779999   46.250000   
           ZTS      101.811356  101.980003  102.400002  100.639999   

Attributes                Open     Volume  Return10  F_return10  Adj Close_fd  \
Date       Symbols                                                              
2019-04-04 XYL       79.660004   637700.0  0.009464    0.036625      1.511516   
           YUM      100.580002   860700.0  0.012703    0.028571      2.359287   
           ZBH      126.570000   817000.0 -0.021304   -0.051833      0.398170   
           ZION      46.250000  2259600.0  0.035396   -0.007970      0.435328   
           ZTS      102.190002  2475500.0  0.023485   -0.027319      2.203596   

Attributes          Adj Close_fd_ewm60  Adj Close_fd_ewm10  \
Date       Symbols                                           
2019-04-04 XYL                0.933151            1.480530   
           YUM                1.940510            2.330753   
           ZBH                0.056535            0.518980   
           ZION               0.274891            0.107368   
           ZTS                1.426573            2.260059   

Attributes          Adj Close_fd_ewm26  Adj Close_fd_ewm12  \
Date       Symbols                                           
2019-04-04 XYL                1.313283            1.459355   
           YUM                2.199734            2.317235   
           ZBH                0.429642            0.518611   
           ZION               0.199046            0.101015   
           ZTS                1.937719            2.215323   

Attributes          Adj Close_fd_sma60  Adj Close_fd_sma10  \
Date       Symbols                                           
2019-04-04 XYL                0.954416            1.330817   
           YUM                1.865014            2.288303   
           ZBH               -0.051455            0.527663   
           ZION               0.469907           -0.150486   
           ZTS                1.193332            2.111457   

Attributes          Adj Close_fd_std60  Adj Close_fd_std10  Adj Close_fd_macd  
Date       Symbols                                                             
2019-04-04 XYL                0.666717            0.620963           0.146071  
           YUM                0.409085            0.131314           0.117501  
           ZBH                0.928333            0.108480           0.088969  
           ZION               0.483390            0.490978          -0.098031  
           ZTS                1.142872            0.541230           0.277604

In [0]:
data_norm.to_pickle("trans_stock_data.pkl")

file = drive.CreateFile()
file.SetContentFile("trans_stock_data.pkl")
file.Upload() 